In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
data = pd.read_csv('rawdata/2014.txt', sep = ';')

/Users/adamlevin/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (20,21,22,27,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.columns = np.array([str(i).strip() for i in list(data.columns.values)]) #remove space in col names

In [4]:
data.columns

Index([u'Tour Code', u'Tour Description', u'Year', u'Tourn.#', u'Player #',
       u'Course #', u'Permanent Tournament #', u'Player First Name',
       u'Player Last Name', u'Round', u'Tournament Name', u'Course Name',
       u'Hole', u'Hole Score', u'Par Value', u'Yardage', u'Shot',
       u'Shot Type(S/P/D)', u'# of Strokes', u'From Location(Scorer)',
       u'From Location(Enhanced)', u'To Location(Laser)',
       u'To Location(Enhanced)', u'Distance', u'Distance to Pin',
       u'In the Hole Flag', u'Around the Green Flag', u'1st Putt Flag',
       u'Distance to Hole after the Shot', u'Time', u'Lie', u'Elevation',
       u'Slope', u'X Coordinate', u'Y Coordinate', u'Z Coordinate',
       u'Distance from Center', u'Distance from Edge', u'Date'],
      dtype='object')

In [5]:
data['X Coordinate'] = [str(i).replace(' ','') for i in data['X Coordinate']] #remove space in coordinates cols
data['Y Coordinate'] = [str(i).replace(' ','') for i in data['Y Coordinate']]
data['Z Coordinate'] = [str(i).replace(' ','') for i in data['Z Coordinate']]

In [6]:
data['X Coordinate'] = [str(i).replace(',','') for i in data['X Coordinate']] #remove commas in coordinates cols
data['Y Coordinate'] = [str(i).replace(',','') for i in data['Y Coordinate']]
data['Z Coordinate'] = [str(i).replace(',','') for i in data['Z Coordinate']]

In [7]:
data['X Coordinate'] = ['-' + str(i)[:len(str(i))-1] if str(i)[len(str(i))-1]=='-' else i for i in data['X Coordinate']] #putting negative in front
data['Y Coordinate'] = ['-' + str(i)[:len(str(i))-1] if str(i)[len(str(i))-1]=='-' else i for i in data['Y Coordinate']]
data['Z Coordinate'] = ['-' + str(i)[:len(str(i))-1] if str(i)[len(str(i))-1]=='-' else i for i in data['Z Coordinate']]

In [8]:
data['X Coordinate'] = pd.to_numeric(data['X Coordinate'])
data['Y Coordinate'] = pd.to_numeric(data['Y Coordinate'])
data['Z Coordinate'] = pd.to_numeric(data['Z Coordinate'])

In [9]:
#unique Course-Round-Hole Tuples
uCRHtps = list(itertools.product(np.unique(data['Course Name']),np.unique(data['Round']),np.unique(data['Hole'])))

In [10]:
from scipy.optimize import fmin_tnc

In [33]:
# coordinates of hole are not given. must be imputed.
# does the distance use the x,y, and z coordinates or just the x and y coordinates?
# test: first find the hole using the x,y, and z coordinates and record the average difference between calculated hole
# location and ball and recorded distance. then do the same using just the x and y coodinates. see which is better.

In [40]:
def f (a):
    x0,y0,z0 = a[0],a[1],a[2]
    return sum((((x-x0)**2 + (y-y0)**2 + (z-z0)**2)**.5-d)**2)/len(x)

def find_the_hole ():
    xopt = fmin_tnc(f,[x0,y0,z0],approx_grad=1)[0].tolist()
    return f(xopt)

In [41]:
#finding the coordinates of the hole
aveds=[]
for u,i in enumerate(uCRHtps):
    if (u+1)%100==0:
#         print u
        print u+1,sum(aveds)/len(aveds)
    subset = data[(data['Course Name']==i[0]) & (data['Round']==int(i[1])) & (data['Hole']==int(i[2])) \
                  & (data['Distance to Hole after the Shot']!=0) & (data['X Coordinate']!=0) & data['Y Coordinate']!=0]
    if subset.shape[0] == 0:
        continue
    d = subset['Distance to Hole after the Shot'].values/12.0
    x = subset['X Coordinate'].values
    y = subset['Y Coordinate'].values
    z = subset['Z Coordinate'].values
    sorted_subset = subset.sort_values('Distance to Hole after the Shot')
    x0 = sorted_subset['X Coordinate'].values[0] ##assume that closest ball recorded to hole does not have an error
    y0 = sorted_subset['Y Coordinate'].values[0]
    z0 = sorted_subset['Z Coordinate'].values[0]
    subset.insert(len(subset.columns),'approx_dist',((x-x0)**2 + (y-y0)**2 + (z-z0)**2)**.5)
    # dist_diff is difference between recorded distance and distance approximated from location of closest recorded shot
    subset.insert(len(subset.columns),'dist_diff', \
                  np.absolute(subset['approx_dist'].values - subset['Distance to Hole after the Shot'].values/12.0))
    before = subset.shape[0]
    # remove very inconsistant shots from record which are likely mistakes
    subset = subset[subset['dist_diff']<sorted_subset['Distance to Hole after the Shot'].values[0]]
    after = subset.shape[0]
    if before-after>0:
        print u, before-after
    d = subset['Distance to Hole after the Shot'].values/12.0
    x = subset['X Coordinate'].values
    y = subset['Y Coordinate'].values
    z = subset['Z Coordinate'].values
    a = find_the_hole()
    aveds.append(a)
print sum(aveds)/len(aveds)

21 1
32 2
33 1
47 1
68 1
69 2
81 1
100 0.0242321226408
99 1
117 1
140 1
152 1
157 1
175 1
200 0.0237952704126
226 1
229 1
244 1
251 1
265 2
280 1
283 3
291 1
293 2
295 1
296 1
300 0.023634517231
306 1
313 1
333 1
347 1
349 1
382 2
400 0.0236467310384
422 2
500 0.0236602997279
600 0.0237141690068
649 1
655 1
658 1
660 1
663 2
677 1
680 1
681 1
692 1
700 0.0236141313418
699 1
713 1
716 1
717 1
730 2
738 5
742 1
744 3
756 3
762 3
774 1
778 2
780 1
792 3
793 3
795 2
796 1
797 2
800 0.0236267296204
799 1
800 4
801 1
803 1
805 1
810 3
815 3
818 1
823 1
824 5
828 1
829 5
832 1
833 3
835 3
841 3
846 1
851 3
855 2
862 1
900 0.0237316132111
1000 0.0237316132111
1008 2
1020 1
1022 1
1032 1
1042 1
1045 1
1081 2
1092 1
1100 0.0238392102474
1101 1
1117 2
1128 2
1145 1
1200 0.0238701370125
1230 2
1248 1
1249 1
1283 1
1298 2
1300 0.0239203553744
1310 1
1370 2
1391 1
1400 0.0237858446115
1435 1
1443 1
1451 1
1452 1
1456 1
1461 1
1469 1
1497 2
1500 0.0237866623818
1525 1
1552 2
1558 1
1576 1
1594 1
1600

In [42]:
def f (a):
    x0,y0 = a[0],a[1]
    return sum((((x-x0)**2 + (y-y0)**2)**.5-d)**2)/len(x)

def find_the_hole ():
    xopt = fmin_tnc(f,[x0,y0],approx_grad=1)[0].tolist()
    return f(xopt)

In [43]:
aveds=[]
for u,i in enumerate(uCRHtps):
    if (u+1)%100==0:
#         print u
        print u+1,sum(aveds)/len(aveds)
    subset = data[(data['Course Name']==i[0]) & (data['Round']==int(i[1])) & (data['Hole']==int(i[2])) \
                  & (data['Distance to Hole after the Shot']!=0) & (data['X Coordinate']!=0) & data['Y Coordinate']!=0]
    if subset.shape[0] == 0:
        continue
    d = subset['Distance to Hole after the Shot'].values/12.0
    x = subset['X Coordinate'].values
    y = subset['Y Coordinate'].values
    sorted_subset = subset.sort_values('Distance to Hole after the Shot')
    x0 = sorted_subset['X Coordinate'].values[0] ##assume that closest ball recorded to hole does not have an error
    y0 = sorted_subset['Y Coordinate'].values[0]
    subset.insert(len(subset.columns),'approx_dist',((x-x0)**2 + (y-y0)**2)**.5)
    # dist_diff is difference between recorded distance and distance approximated from location of closest recorded shot
    subset.insert(len(subset.columns),'dist_diff', \
                  np.absolute(subset['approx_dist'].values - subset['Distance to Hole after the Shot'].values/12.0))
    before = subset.shape[0]
    # remove very inconsistant shots from record which are likely mistakes
    subset = subset[subset['dist_diff']<sorted_subset['Distance to Hole after the Shot'].values[0]]
    after = subset.shape[0]
    if before-after>0:
        print u, before-after
    d = subset['Distance to Hole after the Shot'].values/12.0
    x = subset['X Coordinate'].values
    y = subset['Y Coordinate'].values
    a = find_the_hole()
    aveds.append(a)
print sum(aveds)/len(aveds)

21 1
32 2
33 1
47 1
57 1
68 1
69 2
80 7
81 1
100 0.0256391379818
99 1
117 1
140 1
152 1
157 1
161 4
175 1
179 7
200 0.025554530536
224 4
226 1
229 1
233 1
242 5
244 1
251 1
260 6
265 3
278 3
280 1
283 3
291 1
292 1
293 5
295 1
296 1
300 0.0272396264516
306 1
309 13
313 1
327 3
333 1
347 2
349 1
382 2
400 0.0306047166194
422 2
428 19
500 0.0323804367743
505 62
519 9
523 73
541 1
543 1
559 66
561 1
573 9
600 0.0331033993527
649 1
655 1
658 1
660 1
663 2
677 1
680 1
681 1
692 1
700 0.0312535668032
699 1
713 1
716 1
717 1
730 2
738 5
742 1
744 3
756 3
762 3
774 1
778 2
780 1
792 3
793 3
795 2
796 1
797 2
800 0.0304185736322
799 1
800 4
801 1
803 1
805 1
810 3
815 3
818 1
823 1
824 5
828 2
829 5
832 1
833 3
835 3
839 1
841 3
846 1
851 3
855 2
862 1
900 0.0299210188316
1000 0.0299210188316
1008 2
1019 4
1020 1
1022 1
1024 1
1032 1
1042 1
1045 1
1078 1
1081 2
1092 1
1100 0.0296329955367
1101 1
1117 2
1120 1
1128 2
1145 1
1200 0.0292538436864
1230 2
1240 1
1248 1
1249 1
1274 4
1283 1
1290 1
12

In [47]:
# based on results of number of points that have implausible coordinates, and average error between the recorded dist.
# and this dist. calculated with the imputed coordinates of the hole, it is clear that the recorded distances use 
# the x,y, and z coordinates.
# now recreate dataframe with concatonation of all subsets - without bunk records and with the x,y, and z coordinates
# of the hole.

In [46]:
def f (a):
    x0,y0,z0 = a[0],a[1],a[2]
    return sum((((x-x0)**2 + (y-y0)**2 + (z-z0)**2)**.5-d)**2)/len(x)

def find_the_hole ():
    xopt = fmin_tnc(f,[x0,y0,z0],approx_grad=1)[0].tolist()
    return xopt

In [70]:
# initializing new data frame with two rows which will be deleted after
newdata = pd.DataFrame(data.loc[1:2,:])

#finding the coordinates of the hole and recording the results
for u,i in enumerate(uCRHtps[0:5]):
#     if (u+1)%100==0:
#         print u
#         print newdata.shape
    subset = data[(data['Course Name']==i[0]) & (data['Round']==int(i[1])) & (data['Hole']==int(i[2])) \
                  & (data['Distance to Hole after the Shot']!=0) & (data['X Coordinate']!=0) & \
                  (data['Y Coordinate']!=0)]
    subset_with_holeouts = data[(data['Course Name']==i[0]) & (data['Round']==int(i[1])) & (data['Hole']==int(i[2])) \
                  & (~((data['Distance to Hole after the Shot']!=0) & \
                       ((data['X Coordinate']==0) | (data['Y Coordinate']==0))))]
    print subset.shape
    print subset_with_holeouts.shape
    if subset.shape[0] == 0:
        continue   
    d = subset['Distance to Hole after the Shot'].values/12.0
    x = subset['X Coordinate'].values
    y = subset['Y Coordinate'].values
    z = subset['Z Coordinate'].values
    sorted_subset = subset.sort_values('Distance to Hole after the Shot')
    x0 = sorted_subset['X Coordinate'].values[0] ##assume that closest ball recorded to hole does not have an error
    y0 = sorted_subset['Y Coordinate'].values[0]
    z0 = sorted_subset['Z Coordinate'].values[0]
    subset.insert(len(subset.columns),'approx_dist',((x-x0)**2 + (y-y0)**2 + (z-z0)**2)**.5)
    # dist_diff is difference between recorded distance and distance approximated from location of closest recorded shot
    subset.insert(len(subset.columns),'dist_diff', \
                  np.absolute(subset['approx_dist'].values - subset['Distance to Hole after the Shot'].values/12.0))
    # remove very inconsistant shots from record which are likely mistakes
    subset = subset[subset['dist_diff']<sorted_subset['Distance to Hole after the Shot'].values[0]]
    d = subset['Distance to Hole after the Shot'].values/12.0
    x = subset['X Coordinate'].values
    y = subset['Y Coordinate'].values
    z = subset['Z Coordinate'].values
    a = find_the_hole()
    subset.drop('approx_dist',axis=1,inplace=True)
    subset.drop('dist_diff',axis=1,inplace=True)
    subset.insert(len(subset.columns),'Hole X Coordinate',pd.Series([a[0]]*subset.shape[1]))
    subset.insert(len(subset.columns),'Hole Y Coordinate',pd.Series([a[1]]*subset.shape[1]))
    subset.insert(len(subset.columns),'Hole Z Coordinate',pd.Series([a[2]]*subset.shape[1]))
#     newdata = newdata.append(subset)
    

(385, 39)
(505, 39)
(248, 39)
(368, 39)
(394, 39)
(514, 39)
(436, 39)
(556, 39)
(358, 39)
(478, 39)
